In [1]:
import logging
import logging.handlers
import os
import io
import pandas as pd
import numpy as np
import requests
import json
from chessdotcom import get_player_profile, get_player_stats, get_player_game_archives
import chess.pgn
from converter.pgn_data import PGNData
import time
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [2]:
"""
'./students.json'  should be replaced with database in the future.

That is the goal of this sprint
"""

"\n'./students.json'  should be replaced with database in the future.\n\nThat is the goal of this sprint\n"

In [3]:
def student_df(student_data):
    """
    input:
    student_data - dictionary, where key is the class and value is a list of username of each student
    
    output:
    a dataframe having two columns - class, student class; username, student username
    """
    df = pd.DataFrame({'Keys': list(student_data.keys()), 'Values': list(student_data.values())})
    df = df.explode(column='Values').reset_index(drop=True)
    df.rename(columns = {'Keys':'class', 'Values':'username'}, inplace = True)
    return df

In [4]:
user = {
            "Tianmin Lyu":"tianminlyu"
            }

In [5]:
student_df(user)

,class,username
0,Tianmin Lyu,tianminlyu


In [6]:
def last_n_month(n):
    """
    return the month as yyyy/mm format of the past n months from now
    input -
    n: number of months from past
    
    output -
    a list of month with yyyy/mm format
    """
    months_lst = []
    for num in range(n):
        months = date.today() + relativedelta(months=-num)
        if months.month <= 9:
            months_lst.append(str(months.year) + "/"+ "0" + str(months.month))
        else:
            months_lst.append(str(months.year) + "/"+ str(months.month))
    return months_lst

In [7]:
last_n_month(2)

['2023/09', '2023/08']

In [8]:
def get_user_archives(username, months):
    """
    get archive monthly files of specific chess.com player
    input:
    username - username of the chess.com player
    months - target months that we want to get the archives
    
    output:
    target_month - files of archives according to months parameter
    """
    archives = get_player_game_archives(username).json['archives']
    target_month = []
    for archive in archives:
        if archive[-7:] in months:
            target_month.append(archive)
    return target_month


In [9]:
url = "https://api.chess.com/pub/player/tianminlyu/games/archives"



user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}

r = requests.get(url, headers = user_agent)


In [10]:
archive_data = r.json()


In [12]:
def last_n_month(n):
    """
    purpose:
    return the month as yyyy/mm format of the past n months from now
    
    input -
    n: number of months from past
    
    output -
    a list of month with yyyy/mm format
    """
    months_lst = []
    for num in range(n):
        months = date.today() + relativedelta(months=-num)
        if months.month <= 9:
            months_lst.append(str(months.year) + "/"+ "0" + str(months.month))
        else:
            months_lst.append(str(months.year) + "/"+ str(months.month))
    return months_lst

In [13]:
last_n_month(2)

['2023/09', '2023/08']

In [14]:
def get_user_archives(username, nr_months):
    """
    purpose:
    get archive monthly files of specific chess.com player
    
    input:
    username - username of the chess.com player
    nr_months - integer, nummber of past months that we want to get the archives
    
    output:
    target_month - files of archives according to months parameter
    """
    url = "https://api.chess.com/pub/player/{username}/games/archives".format(username = username)
    user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}
    archive_request = requests.get(url, headers = user_agent)
    archives = archive_request.json()['archives']
    past_months = last_n_month(nr_months)
    target_month = []
    for archive in archives:
        if archive[-7:] in past_months:
            target_month.append(archive)
    return target_month

In [15]:
get_user_archives('tianminlyu',2)

['https://api.chess.com/pub/player/tianminlyu/games/2023/08',
 'https://api.chess.com/pub/player/tianminlyu/games/2023/09']

In [16]:
archive_url = "https://api.chess.com/pub/player/tianminlyu/games/2023/09"

user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}
archive_request = requests.get(archive_url, headers = user_agent)

archive_request

<Response [200]>

In [17]:
def get_archive_games(filename):
    """
    purpose:
    
    return games in one archive file
    
    input:
    filename - filename that contains game urls
    
    output: 
    """
    games = requests.get(filename,headers = user_agent).json()['games']
    return games

In [18]:
for game in archive_request.json()['games']:
    print(datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S'))
    break

2023-09-01 09:39:15


In [19]:
students = ['yaohengli',
           'chessloverma',
           'chengliam',
           'emmaxli',
           'akfunchess66',
           
           'willhanzhu',
           'TLPAWN',
           'Jasminezhao777',
           'Justinzhao777',
           'Milkmilkok',
           'zlicyigloo',
           'Zora_zhu',
           'dogwater1012000',
           
           'AJLinVH',
           'charliezienyang',
           'whatwhywhywhat',
           'ZhouYuanLi',
           'Logicalcheetah26',
           'Nolan330']

students = [x.lower() for x in students]

In [24]:
end_times = []
white_players = []
black_players = []
time_controls = []
urls = []

for student in students:
    print(student.upper())
    archives = get_user_archives(student,2)
    #print(archives)
    for archive in archives[::-1]:
        games = get_archive_games(archive)
        for game in games[::-1]:
            #print(game)
            if (game['white']['username'].lower() == student.lower() and game['black']['username'].lower() in students):
                end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
                print(end_time)
                print("[w]" + student)
                print("[b]" + game['black']['username'])
                print("time control: " + game['time_control'])
                print("          ")
                
                end_times.append(end_time)
                white_players.append(student.lower())
                black_players.append(game['black']['username'].lower())
                time_controls.append(game['time_control'])
                urls.append(game['url'])
                
            elif (game['black']['username'].lower() == student.lower() and game['white']['username'].lower() in students):
                end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
                print(end_time)
                print("[w]" + game['white']['username'])
                print("[b]" + student)
                print("time control: " + game['time_control'])
                print(game['url'])

                print("          ")
                
                end_times.append(end_time)
                white_players.append(game['white']['username'].lower())
                black_players.append(student.lower())
                time_controls.append(game['time_control'])
                urls.append(game['url'])
    print("---------")

YAOHENGLI
2023-09-04 23:33:47
[w]yaohengli
[b]ZhouYuanLi
time control: 900+10
https://www.chess.com/game/live/87607302045
          
2023-09-02 17:37:21
[w]yaohengli
[b]chessloverMA
time control: 900+10
https://www.chess.com/game/live/87414156519
          
2023-08-10 02:48:01
[w]chessloverMA
[b]yaohengli
time control: 900+10
https://www.chess.com/game/live/85373467983
          
---------
CHESSLOVERMA
2023-09-09 17:32:21
[w]chessloverma
[b]akfunchess66
time control: 900+10
https://www.chess.com/game/live/88019431849
          
2023-09-09 17:26:43
[w]akfunchess66
[b]chessloverma
time control: 900+10
https://www.chess.com/game/live/88018925149
          
2023-09-09 16:16:11
[w]Nolan330
[b]chessloverma
time control: 900+10
https://www.chess.com/game/live/88014671171
          
2023-09-09 16:05:10
[w]chessloverma
[b]Nolan330
time control: 600
https://www.chess.com/game/live/88014571429
          
2023-09-04 16:24:29
[w]chessloverma
[b]whatwhywhywhat
time control: 900+10
https://www.chess.

In [21]:
df = pd.DataFrame()
df['end_time'] = end_times
df['white_player'] = white_players
df['black_player'] = black_players
df['time_control'] = time_controls
df['url'] = urls
df = df.sort_values(by = 'end_time', ascending = False)
df = df.drop_duplicates()

In [ ]:
def upload_df(name, df, sheet_url):
    """
    purpose:
    upload df to google sheet RCC_chess_game_result
    each class/csv/file represent one sheet
    
    input - 
    name: class name, sheet tab
    df: df that will be uploaded for each tab
    """
    #spreadsheet_key = '12R6hwzKys_DQE6vFpuOLGpe68hGHktSzd65AkR0nOsA' # sheet url from RCC_chess_game_result
    scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
    creds = ServiceAccountCredentials.from_json_keyfile_name("./creds.json", scope)
    wks_name = name
    df = rp_nan_empty(df)
    d2g.upload(df, sheet_url, wks_name, credentials=creds)  

In [ ]:
#upload_df("2023fall", df, '1YbU3GZq58mWu5Kl4l4gPhq96aohmk8gFxbzGr6cpA7o')

In [ ]:
def game_notification():
    end_times = []
    white_players = []
    black_players = []
    time_controls = []

    for student in students:
        print(student.upper())
        archives = get_user_archives(student,2)
        #print(archives)
        for archive in archives[::-1]:
            games = get_archive_games(archive)
            for game in games[::-1]:
                #print(game)
                if (game['white']['username'].lower() == student.lower() and game['black']['username'].lower() in students):
                    end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
                    print(end_time)
                    print("[w]" + student)
                    print("[b]" + game['black']['username'])
                    print("time control: " + game['time_control'])
                    print("          ")
                    
                    end_times.append(end_time)
                    white_players.append(student.lower())
                    black_players.append(game['black']['username'].lower())
                    time_controls.append(game['time_control'])
                    
                elif (game['black']['username'].lower() == student.lower() and game['white']['username'].lower() in students):
                    end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
                    print(end_time)
                    print("[w]" + game['white']['username'])
                    print("[b]" + student)
                    print("time control: " + game['time_control'])
                    print("          ")
                    
                    end_times.append(end_time)
                    white_players.append(game['white']['username'].lower())
                    black_players.append(student.lower())
                    time_controls.append(game['time_control'])
    print("---------")
    df = pd.DataFrame()
    df['end_time'] = end_times
    df['white_player'] = white_players
    df['black_player'] = black_players
    df['time_control'] = time_controls
    df = df.sort_values(by = 'end_time', ascending = False)
    df = df.drop_duplicates()
    return df
    
game_notification()

In [ ]:
test = [1]

In [ ]:
test[::-1]

In [ ]:
'https://api.chess.com/pub/player/yaohengli/games/2023/08'

In [6]:
import sys

In [7]:
sys.path.insert(1, r'../')

In [8]:
import googlesheet

In [9]:
print(googlesheet.name)

tianmin
